In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from tqdm import tqdm

import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree.export import export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz

from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers

In [2]:
df = pd.read_csv('train.csv', index_col=0)
answer = pd.read_csv('test.csv', index_col=0)

In [3]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

AttributeError: module 'tensorflow' has no attribute 'Session'

In [4]:
df.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


## 데이터 준비

In [5]:
X = df.loc[:, df.columns != "type"]
X = X.loc[:, X.columns != "fiberID"]
y = df["type"]
answer = answer.loc[:, answer.columns != "fiberID"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## 전처리

In [7]:
out = []
for column in list(X_train.columns)[2:]:
    X_train = X_train[X_train[column] < 40] 
    X_train = X_train[X_train[column] > 15]

In [8]:
ilist = X_train.index.values.tolist()
y_train = y_train[ilist]

## RF

In [8]:
forest = RandomForestClassifier(n_estimators=1000, random_state=0)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [9]:
y_pred = forest.predict(X_test)
print('Accuracy =', accuracy_score(y_test, y_pred))

Accuracy = 0.8754125137504584


In [10]:
forest_probas = forest.predict_proba(X_test)

In [11]:
score = sklearn.metrics.log_loss(y_test, forest_probas)
print(score)

0.4137424103974615


## 로지스틱

In [12]:
LR = LogisticRegression(random_state=77)
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=77, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
y_pred = LR.predict(X_test)
print('Accuracy =', accuracy_score(y_test, y_pred))

Accuracy = 0.726957565252175


In [14]:
LR_probas = LR.predict_proba(X_test)
score = sklearn.metrics.log_loss(y_test, LR_probas)
print(score)

0.9174112841691602


## LGB

In [15]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [16]:
lgbm = lgb.LGBMClassifier(num_leaves = 100)

In [17]:
lgbm.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=100, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [18]:
y_pred = lgbm.predict(X_test)
print('Accuracy =', accuracy_score(y_test, y_pred))

Accuracy = 0.7801426714223807


In [19]:
LGBM_probas = lgbm.predict_proba(X_test)
score = sklearn.metrics.log_loss(y_test, LGBM_probas)
print(score)

4.206471224983855


# 옹

In [12]:
submission_probas = forest.predict_proba(answer)
sub = pd.DataFrame(submission_probas)
sub.columns = forest.classes_

In [13]:
forsub = pd.read_csv('sample_submission.csv')
sub = sub.reindex(columns=list(forsub.columns))
sub.to_csv('sub.csv')

## Autokeras

In [ ]:
import autokeras as ak

clf = ak.StructuredDataClassifier()
clf.fit(X_train, y_train)
results = clf.predict(x_test)

Train for 3421 steps, validate for 856 steps
Epoch 1/1000
3421/3421 [==============================] - ETA: 15:44 - loss: 20.0236 - accuracy: 0.0000e+ - ETA: 20s - loss: 6.2496 - accuracy: 0.2441       - ETA: 12s - loss: 4.3710 - accuracy: 0.272 - ETA: 9s - loss: 3.6236 - accuracy: 0.316 - ETA: 7s - loss: 3.2095 - accuracy: 0.33 - ETA: 6s - loss: 2.9573 - accuracy: 0.34 - ETA: 5s - loss: 2.7768 - accuracy: 0.35 - ETA: 5s - loss: 2.6430 - accuracy: 0.36 - ETA: 4s - loss: 2.5326 - accuracy: 0.37 - ETA: 4s - loss: 2.4429 - accuracy: 0.38 - ETA: 4s - loss: 2.3692 - accuracy: 0.38 - ETA: 4s - loss: 2.3082 - accuracy: 0.39 - ETA: 3s - loss: 2.2511 - accuracy: 0.40 - ETA: 3s - loss: 2.1988 - accuracy: 0.40 - ETA: 3s - loss: 2.1560 - accuracy: 0.41 - ETA: 3s - loss: 2.1188 - accuracy: 0.41 - ETA: 3s - loss: 2.0836 - accuracy: 0.41 - ETA: 3s - loss: 2.0531 - accuracy: 0.42 - ETA: 3s - loss: 2.0235 - accuracy: 0.42 - ETA: 3s - loss: 1.9962 - accuracy: 0.42 - ETA: 2s - loss: 1.9691 - accuracy: 0.

Train for 3421 steps, validate for 856 steps
Epoch 1/1000
3421/3421 [==============================] - ETA: 15:23 - loss: 8.4118 - accuracy: 0.062 - ETA: 33s - loss: 3.3840 - accuracy: 0.2051  - ETA: 18s - loss: 2.7503 - accuracy: 0.291 - ETA: 14s - loss: 2.4660 - accuracy: 0.345 - ETA: 12s - loss: 2.3036 - accuracy: 0.371 - ETA: 10s - loss: 2.1938 - accuracy: 0.390 - ETA: 9s - loss: 2.0926 - accuracy: 0.407 - ETA: 9s - loss: 2.0183 - accuracy: 0.41 - ETA: 8s - loss: 1.9583 - accuracy: 0.42 - ETA: 7s - loss: 1.9081 - accuracy: 0.43 - ETA: 7s - loss: 1.8549 - accuracy: 0.44 - ETA: 7s - loss: 1.8152 - accuracy: 0.44 - ETA: 6s - loss: 1.7824 - accuracy: 0.45 - ETA: 6s - loss: 1.7535 - accuracy: 0.45 - ETA: 6s - loss: 1.7296 - accuracy: 0.45 - ETA: 6s - loss: 1.7068 - accuracy: 0.45 - ETA: 5s - loss: 1.6835 - accuracy: 0.46 - ETA: 5s - loss: 1.6608 - accuracy: 0.46 - ETA: 5s - loss: 1.6386 - accuracy: 0.46 - ETA: 5s - loss: 1.6175 - accuracy: 0.47 - ETA: 5s - loss: 1.5961 - accuracy: 0.47 